# 사용데이터
https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=548

In [ ]:
import os
import shutil
from PIL import Image
from tqdm import tqdm
import xml.etree.ElementTree as ET

In [ ]:
def PointStr2List(points):
    points = points.split(';')
    ans = []
    for pnt in points:
        x,y = pnt.split(',')
        x = int(round(float(x)))
        y = int(round(float(y)))
        ans.append([x,y])
    return ans


def FindXmymxgyg(points, size):
    X,Y = [],[]
    width, height = size
    for point in points:
        x,y = point
        X.append(x)
        Y.append(y)
    x1 = min(X)
    y1 = min(Y)
    x2 = max(X)
    y2 = max(Y)
    xm = (x1+x2)/2 /width 
    ym = (y1+y2)/2 /height
    xg = (x2-x1) /width  *2
    yg = (y2-y1) /height *4
    xm = round(xm, 6)
    ym = round(ym, 6)
    xg = round(xg, 6)
    yg = round(yg, 6)
    return xm,ym,xg,yg


def PointEnt2Crop(points, xmymxgyg, size):
    ans = []
    width, height = size
    xm, ym, xg, yg = xmymxgyg
    xm, ym, xg, yg = xm *width, ym *height, xg *width, yg *height
    for pnt in points:
        x,y = pnt
        x = round((x -(xm-xg/2))/xg, 6)
        y = round((y -(ym-yg/2))/yg, 6)
        ans.append([x,y]) 
    return ans


def CropAndSaveImgXmymxgyg(img_path, xmymxgyg, save_path):
    image = Image.open(img_path)
    xm, ym, xg, yg = xmymxgyg
    width, height = image.size
    x1 = int(round((xm - xg/2) * width))
    x2 = int(round((xm + xg/2) * width))
    y1 = int(round((ym - yg/2) * height))
    y2 = int(round((ym + yg/2) * height))
    cropped_image = image.crop((x1, y1, x2, y2))
    cropped_image.save(save_path)
    

def SaveLabel2Txt(center_point, lid_polygon, save_path):
    points = center_point + lid_polygon
    with open(save_path, 'w') as file:
        file.write('0 0.500000 0.500000 0.750000 0.500000 ')
        for x,y in points:
            content = f'{x} {y} '
            file.write(content)

In [ ]:
i = 0

mod = ['train', 'val']
mod = mod[i]

image_root = f'./imaging'
label_root = f'./{mod}_labeling'
save_image_root = f'./pos_data/{mod}/images'
save_label_root = f'./pos_data/{mod}/labels'


img_set = set(os.listdir(image_root))

with tqdm(total = len(os.listdir(label_root))) as pbar :
    for filename in os.listdir(label_root):
        xml_path = os.path.join(label_root, filename)
        tree = ET.parse(xml_path)
        root_element = tree.getroot()
        image_elements = root_element.findall('image')
        
        for image_element in image_elements:
            # <image> 엘리먼트의 name 속성 추출
            image_name = image_element.get('name')
            
            if image_name not in img_set:
                continue
            
            image_width = float(image_element.get('width'))
            image_height = float(image_element.get('height'))
            image_size = (image_width, image_height)

            points_elements = image_element.findall('points')
            for point in points_elements:
                if   point.get('label') == "left_center":
                    LC_pnt = point.get('points')
                    LC_pnt = PointStr2List(LC_pnt)
                elif point.get('label') == "right_center":
                    RC_pnt = point.get('points')
                    RC_pnt = PointStr2List(RC_pnt)

            polygons_elements = image_element.findall('polygon')
            for polygon in polygons_elements:
                if   polygon.get('label') == "left_eyelid":
                    LD_pnt = polygon.get('points')
                    LD_pnt = PointStr2List(LD_pnt)
                elif polygon.get('label') == "right_eyelid":
                    RD_pnt = polygon.get('points')
                    RD_pnt = PointStr2List(RD_pnt)
            
            L_xmymxgyg = FindXmymxgyg(LD_pnt, image_size)
            R_xmymxgyg = FindXmymxgyg(RD_pnt, image_size)
            
            img_path = os.path.join(image_root, image_name)
            L_img_save = save_image_root +'/'+ image_name[:-4] +'_0.jpg'
            R_img_save = save_image_root +'/'+ image_name[:-4] +'_1.jpg'
            CropAndSaveImgXmymxgyg(img_path, L_xmymxgyg, L_img_save)
            CropAndSaveImgXmymxgyg(img_path, R_xmymxgyg, R_img_save)
            
            T_LC_pnt = PointEnt2Crop(LC_pnt, L_xmymxgyg, image_size)
            T_LD_pnt = PointEnt2Crop(LD_pnt, L_xmymxgyg, image_size)
            T_RC_pnt = PointEnt2Crop(RC_pnt, R_xmymxgyg, image_size)
            T_RD_pnt = PointEnt2Crop(RD_pnt, R_xmymxgyg, image_size)
            L_lbl_save = save_label_root +'/'+ image_name[:-4] +'_0.txt'
            R_lbl_save = save_label_root +'/'+ image_name[:-4] +'_1.txt'
            SaveLabel2Txt(T_LC_pnt, T_LD_pnt, L_lbl_save)
            SaveLabel2Txt(T_RC_pnt, T_RD_pnt, R_lbl_save)
    
        pbar.update(1)
        
print('Done...!')